<a href="https://colab.research.google.com/github/SreejaThumma/DL_assignment2/blob/main/DL_Q1_A2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

def load_data(path):
    df = pd.read_csv(path, sep="\t", header=None, names=["target", "input", "freq"])
    df = df.dropna()
    return df["input"].astype(str).tolist(), df["target"].astype(str).tolist()

train_inputs, train_targets = load_data("/content/drive/MyDrive/hi.translit.sampled.train.tsv")
dev_inputs, dev_targets = load_data("/content/drive/MyDrive/hi.translit.sampled.dev.tsv")
test_inputs, test_targets = load_data("/content/drive/MyDrive/hi.translit.sampled.test.tsv")

# Add start (\t) and end (\n) tokens
train_targets = ['\t' + t + '\n' for t in train_targets]
dev_targets = ['\t' + t + '\n' for t in dev_targets]
test_targets = ['\t' + t + '\n' for t in test_targets]


In [ ]:
input_characters = sorted(set(''.join(train_inputs + dev_inputs + test_inputs)))
target_characters = sorted(set(''.join(train_targets + dev_targets + test_targets)))

input_token_index = {char: i + 1 for i, char in enumerate(input_characters)}  # 0 for padding
target_token_index = {char: i + 1 for i, char in enumerate(target_characters)}
reverse_target_index = {i: char for char, i in target_token_index.items()}

num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import tensorflow as tf

def encode_sequences(inputs, targets, input_index, target_index, max_encoder_len, max_decoder_len):
    encoder_input = [[input_index.get(c, 0) for c in seq] for seq in inputs]
    decoder_input = [[target_index.get(c, 0) for c in seq[:-1]] for seq in targets]
    decoder_output = [[target_index.get(c, 0) for c in seq[1:]] for seq in targets]

    encoder_input = pad_sequences(encoder_input, maxlen=max_encoder_len, padding='post')
    decoder_input = pad_sequences(decoder_input, maxlen=max_decoder_len-1, padding='post')
    decoder_output = pad_sequences(decoder_output, maxlen=max_decoder_len-1, padding='post')

    return encoder_input, decoder_input, tf.keras.utils.to_categorical(decoder_output, num_classes=num_decoder_tokens)

max_encoder_seq_length = max(len(seq) for seq in train_inputs)
max_decoder_seq_length = max(len(seq) for seq in train_targets)

train_enc, train_dec_in, train_dec_out = encode_sequences(train_inputs, train_targets, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)
dev_enc, dev_dec_in, dev_dec_out = encode_sequences(dev_inputs, dev_targets, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)
test_enc, test_dec_in, test_dec_out = encode_sequences(test_inputs, test_targets, input_token_index, target_token_index, max_encoder_seq_length, max_decoder_seq_length)


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense

def build_seq2seq_model(embedding_dim=128, hidden_units=256):
    encoder_inputs = Input(shape=(None,))
    encoder_embedding = Embedding(input_dim=num_encoder_tokens, output_dim=embedding_dim)(encoder_inputs)
    encoder_outputs, state_h, state_c = LSTM(hidden_units, return_state=True)(encoder_embedding)
    encoder_states = [state_h, state_c]

    decoder_inputs = Input(shape=(None,))
    decoder_embedding = Embedding(input_dim=num_decoder_tokens, output_dim=embedding_dim)(decoder_inputs)
    decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
    decoder_dense = Dense(num_decoder_tokens, activation='softmax')
    decoder_outputs = decoder_dense(decoder_outputs)

    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

    return model


In [ ]:
model = build_seq2seq_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    [train_enc, train_dec_in], train_dec_out,
    validation_data=([dev_enc, dev_dec_in], dev_dec_out),
    batch_size=64,
    epochs=30
)


Epoch 1/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 9s 10ms/step - accuracy: 0.7091 - loss: 1.2141 - val_accuracy: 0.7812 - val_loss: 0.7841
Epoch 2/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.8021 - loss: 0.6912 - val_accuracy: 0.8810 - val_loss: 0.3862
Epoch 3/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.8939 - loss: 0.3455 - val_accuracy: 0.9173 - val_loss: 0.2688
Epoch 4/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.9260 - loss: 0.2381 - val_accuracy: 0.9299 - val_loss: 0.2242
Epoch 5/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9412 - loss: 0.1900 - val_accuracy: 0.9378 - val_loss: 0.1978
Epoch 6/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.9509 - loss: 0.1583 - val_accuracy: 0.9407 - val_loss: 0.1906
Epoch 7/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.9577 - loss: 0.1361 - val_accuracy: 0.9420 - val_loss: 0.1849
Epoch 8/30
691/691 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.9635 - loss: 0.1170 - val_acc

In [ ]:
train_loss, train_acc = model.evaluate([train_enc, train_dec_in], train_dec_out, verbose=0)
test_loss, test_acc = model.evaluate([test_enc, test_dec_in], test_dec_out, verbose=0)

print(f"\n✅ Train Accuracy: {train_acc * 100:.2f}%")
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")



✅ Train Accuracy: 99.42%
✅ Test Accuracy: 94.35%


In [ ]:
# Encoder inference
encoder_model = Model(model.input[0], model.layers[4].output[1:])

# Decoder inference
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_single = Input(shape=(1,))
decoder_embedding_inf = model.layers[3](decoder_inputs_single)
decoder_outputs, state_h, state_c = model.layers[5](decoder_embedding_inf, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = model.layers[6](decoder_outputs)

decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)


In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = target_token_index['\t']
    decoded_sentence = ''
    while True:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_index.get(sampled_token_index, '')
        if sampled_char == '\n' or sampled_char == '':
            break
        decoded_sentence += sampled_char
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]
    return decoded_sentence


In [ ]:
print("\n🔍 Sample Predictions:")
correct = 0
for i in range(20):
    input_text = test_inputs[i]
    input_seq = pad_sequences([[input_token_index.get(c, 0) for c in input_text]], maxlen=max_encoder_seq_length, padding='post')
    decoded = decode_sequence(input_seq)
    actual = test_targets[i][1:-1]
    print(f"Input: {input_text} => Predicted: {decoded} | Actual: {actual}")
    if decoded == actual:
        correct += 1
print(f"\n🎯 Accuracy on 20 samples: {correct / 20:.2f}")



🔍 Sample Predictions:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Input: ank => Predicted: अंक | Actual: अंक
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step
Input: anka => Predicted: आंका | Actual: अंक
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Input: ankit => Predicted: आंकित | Actual: अंकित
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


In [ ]:
train_loss, train_acc = model.evaluate([train_enc, train_dec_in], train_dec_out, verbose=0)
test_loss, test_acc = model.evaluate([test_enc, test_dec_in], test_dec_out, verbose=0)

print(f"\n✅ Train Accuracy: {train_acc * 100:.2f}%")
print(f"✅ Test Accuracy: {test_acc * 100:.2f}%")


✅ Train Accuracy: 99.42%
✅ Test Accuracy: 94.35%
